# 1.Set Up the YOLOv5 Pipeline

## Install dependencies

In [1]:
!pip install torch torchvision torchaudio
!pip install ultralytics


## Clone the YOLOv5 repository and set up the environment

In [2]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


f:\Assessment\Custom_Object_Detection_and_Novel_Bounding_Box_Metric_with_YOLO\yolov5


Cloning into 'yolov5'...


# 2.Dataset Preparation

## Download Dataset 
[about ~50–100 images with bounding box annotations is enough for a 
proof-of-concept. ]

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6PdYCi4jBGkqWpGKkaGi")
project = rf.workspace("project-3vvpk").project("dog-and-cat-tmn5b")
version = project.version(2)
dataset = version.download("yolov5")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dog-and-cat-2 in yolov5pytorch:: 100%|██████████| 204/204 [00:00<00:00, 1703.50it/s]


## Setup the data.yaml file with the correct dataset path (after the download)

# 3.Train YOLOv5 on Your Dataset

## Train YOLOv5 on the dataset

In [5]:
!python train.py --img 640 --batch 16 --epochs 50 --data F:/Assessment/Custom_Object_Detection_and_Novel_Bounding_Box_Metric_with_YOLO/dog-and-cat-2/data.yaml --weights yolov5s.pt


wandb: WARNING  wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
train: weights=yolov5s.pt, cfg=, data=F:/Assessment/Custom_Object_Detection_and_Novel_Bounding_Box_Metric_with_YOLO/dog-and-cat-2/data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndj

# 4. Define a Custom Bounding Box Similarity Metric


In this step, we define a custom similarity metric for bounding boxes, focusing on the Intersection over Union (IoU) as a standard measure for evaluating the overlap between two bounding boxes.

### What is IoU (Intersection over Union)?

IoU is a widely used metric for evaluating the similarity between two bounding boxes. It is defined as the ratio of the area of overlap between two bounding boxes to the area of their union.

### Formula:

The formula for IoU is:

$$
IoU = \frac{\text{Area of Overlap}}{\text{Area of Union}}
$$

Where:

- **Area of Overlap**: The area where the two bounding boxes overlap.
- **Area of Union**: The total area covered by both bounding boxes, including their overlap.

### Steps to Calculate IoU:

1. **Calculate the Area of Overlap**: 
   - Find the intersection of the two bounding boxes.
   
2. **Calculate the Area of Union**:
   - Find the total area covered by both bounding boxes combined.

3. **Compute IoU**:
   - Divide the area of overlap by the area of union to get the IoU score.

In [6]:
import numpy as np
import torch

def custom_bbox_similarity(box1, box2):
    """
    Compute a custom bounding box similarity score.
    Inputs:
        - box1, box2: Bounding boxes in [x1, y1, x2, y2] format
    Returns:
        - similarity score (0 to 1)
    """
    # Compute IoU
    def compute_iou(box1, box2):
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection

        return intersection / union if union > 0 else 0

    iou = compute_iou(box1, box2)

    # Compute center distance penalty
    center1 = [(box1[0] + box1[2]) / 2, (box1[1] + box1[3]) / 2]
    center2 = [(box2[0] + box2[2]) / 2, (box2[1] + box2[3]) / 2]
    center_dist = np.linalg.norm(np.array(center1) - np.array(center2))
    center_penalty = np.exp(-center_dist / 50)  # Normalize by 50 pixels

    # Compute aspect ratio similarity
    def aspect_ratio(box):
        return (box[2] - box[0]) / (box[3] - box[1]) if (box[3] - box[1]) > 0 else 1

    aspect_ratio_similarity = 1 - abs(aspect_ratio(box1) - aspect_ratio(box2)) / max(aspect_ratio(box1), aspect_ratio(box2))

    # Weighted combination
    similarity_score = (iou + center_penalty + aspect_ratio_similarity) / 3
    return similarity_score


# 5. Integrate Metric into Training or Evaluation

In [10]:
!python val.py --data F:/Assessment/Custom_Object_Detection_and_Novel_Bounding_Box_Metric_with_YOLO/dog-and-cat-2/data.yaml --weights runs/train/exp/weights/best.pt


Custom Metric (Avg Similarity Score) for batch 0: 0.2995
Custom Metric (Avg Similarity Score) for batch 1: 0.4495
Custom Metric (Avg Similarity Score) for batch 2: 0.2714
Custom Metric (Avg Similarity Score) for batch 3: 0.2636
Custom Metric (Avg Similarity Score) for batch 4: 0.2649
Custom Metric (Avg Similarity Score) for batch 5: 0.2936
Custom Metric (Avg Similarity Score) for batch 6: 0.3767
Custom Metric (Avg Similarity Score) for batch 7: 0.3059
Custom Metric (Avg Similarity Score) for batch 8: 0.2962
Custom Metric (Avg Similarity Score) for batch 9: 0.3266
Custom Metric (Avg Similarity Score) for batch 10: 0.4677
Custom Metric (Avg Similarity Score) for batch 11: 0.3493
Custom Metric (Avg Similarity Score) for batch 12: 0.3336


val: data=F:/Assessment/Custom_Object_Detection_and_Novel_Bounding_Box_Metric_with_YOLO/dog-and-cat-2/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  v7.0-398-g5cdad892 Python-3.9.20 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

val: Scanning F:\Assessment\Custom_Object_Detection_and_Novel_Bounding_Box_Metric_with_YOLO\dog-and-cat-2\valid\labels.cache... 13 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<?, ?it/s]
val: Scanning F:\Assessment\Custom_Object_Detection_and_Novel_Bounding_Box_Metric_with_YOLO\dog-and-cat-2\valid\labels.cache... 13 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<?, ?i